In [ ]:
#The following script is supposed to predict next ten stock prices

import tensorflow as tf
import pandas as pd
import numpy as np

def xy_data(dataset):
    X_train=[]
    y_train=[]
    T = 5
    for t in range(len(dataset)-T):
        X_train.append(dataset[t:t+T])
        y_train.append(dataset[t+T])
    return X_train,y_train

def create_model(X_train,y_train):
    X_train = np.array(X_train).reshape(-1,5,1)
    y_train = np.array(y_train)
    # print(X_train.shape)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(5))
    model.add(tf.keras.layers.Dense(1,activation = 'relu'))
    model.compile(loss='mse',optimizer = tf.keras.optimizers.Adam(learning_rate=0.05))
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',patience=20,restore_best_weights=True)
    n = list(X_train.shape)[0]
    model.fit(X_train[int(0.2*n):],y_train[int(0.2*n):],epochs=200,validation_data=(X_train[:int(0.2*n)],y_train[:int(0.2*n)]),steps_per_epoch=5,callbacks=[callback])
    return model
    
def predict(model,X):
    pred = []
    for _ in range(12):
        # print(X[-5:])
        data = np.array(X[-5:]).reshape(1,5,1)
        # print(data.shape)
        p = float(model.predict(data))
        X.append(p)
        pred.append(p)
    return pred
    
def get_pred(Ticker,dataset):
    ind = list(dataset['Ticker']).index(Ticker) 
    X_train,y_train = xy_data(list(dataset.transpose()[ind])[1:])
    model = create_model(X_train,y_train)
    predict_year = predict(model,list(dataset.transpose()[ind])[1:])
    return predict_year

def get_dict(preds):
    y=2022
    m=4
    dates = {}
    for i in range(12):
        dates[f'{y}-{m}-01'] = preds[i]
        if m==12:
            m = 1
            y +=1
        else:
            m+=1
    return dates
                                                             
stockdata = pd.read_csv("FILE PATH HERE")

try:
    preds = get_pred('twou',stockdata)
except:
    preds=[]
    print('Ticker not found')

dict_pred = get_dict(preds)